In [2]:
!wget 'https://docs.google.com/uc?export=download&id=142PkGZFvwo7v3sm0eSEi49oLXk6gtSOj' -O olist_customers_dataset.csv
!wget 'https://docs.google.com/uc?export=download&id=1HV-KlQmF8me1jDb_Ch2bGFUZqZUxNbkF' -O olist_geolocation_dataset.csv
!wget 'https://docs.google.com/uc?export=download&id=12yGLKf3lP-BVZMVan310YU6QktmqUeNP' -O olist_order_items_dataset.csv
!wget 'https://docs.google.com/uc?export=download&id=153SSO1jupTRDxDnlpMqZ9zPm400pZBE7' -O olist_order_payments_dataset.csv
!wget 'https://docs.google.com/uc?export=download&id=1fWIOYCiTfvZ-ON5479AkCPBuzRksOC4G' -O olist_order_reviews_dataset.csv
!wget 'https://docs.google.com/uc?export=download&id=18Z0MwFTJILcXWpUXOB2j9xBfayte8Dvd' -O olist_orders_dataset.csv
!wget 'https://docs.google.com/uc?export=download&id=1vqluVCXG8B1ZHiXhcKwqqoQggtgdsyuk' -O olist_products_dataset.csv
!wget 'https://docs.google.com/uc?export=download&id=1dr7ttKMXjlbgY7lWm-LLjcbd0ht5mDpU' -O olist_sellers_dataset.csv
!wget 'https://docs.google.com/uc?export=download&id=18bKHrhd1gVOmz5xtfRO0MDt23RL_Tojv' -O product_category_name_translation.csv

--2025-09-02 08:17:08--  https://docs.google.com/uc?export=download&id=142PkGZFvwo7v3sm0eSEi49oLXk6gtSOj
Resolving docs.google.com (docs.google.com)... 192.178.129.139, 192.178.129.113, 192.178.129.138, ...
Connecting to docs.google.com (docs.google.com)|192.178.129.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=142PkGZFvwo7v3sm0eSEi49oLXk6gtSOj&export=download [following]
--2025-09-02 08:17:08--  https://drive.usercontent.google.com/download?id=142PkGZFvwo7v3sm0eSEi49oLXk6gtSOj&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 64.233.179.132, 2607:f8b0:4001:c01::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|64.233.179.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9033957 (8.6M) [application/octet-stream]
Saving to: ‘olist_customers_dataset.csv’

olist_customers_dat 100%[===================>]   8.6

In [3]:
import pandas as pd

# Carregar os datasets
orders = pd.read_csv('olist_orders_dataset.csv')
order_items = pd.read_csv('olist_order_items_dataset.csv')
products = pd.read_csv('olist_products_dataset.csv')
geolocation = pd.read_csv('olist_geolocation_dataset.csv')

# Visualizar as primeiras linhas
print(orders.head())
print(order_items.head())
print(products.head())
print(geolocation.head())

                           order_id                       customer_id  \
0  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
1  53cdb2fc8bc7dce0b6741e2150273451  b0830fb4747a6c6d20dea0b8c802d7ef   
2  47770eb9100c2d0c44946d9cf07ec65d  41ce2a54c0b03bf3443c3d931a367089   
3  949d5b44dbf5de918fe9c16f97b45f8a  f88197465ea7920adcdbec7375364d82   
4  ad21c59c0840e6cb83a9ceb5573f8159  8ab97904e6daea8866dbdbc4fb7aad2c   

  order_status order_purchase_timestamp    order_approved_at  \
0    delivered      2017-10-02 10:56:33  2017-10-02 11:07:15   
1    delivered      2018-07-24 20:41:37  2018-07-26 03:24:27   
2    delivered      2018-08-08 08:38:49  2018-08-08 08:55:23   
3    delivered      2017-11-18 19:28:06  2017-11-18 19:45:59   
4    delivered      2018-02-13 21:18:39  2018-02-13 22:20:29   

  order_delivered_carrier_date order_delivered_customer_date  \
0          2017-10-04 19:55:00           2017-10-10 21:25:13   
1          2018-07-26 14:31:00           2018-08

In [18]:
import pandas as pd
import plotly.express as px

In [24]:
# Carregar datasets
orders = pd.read_csv('olist_orders_dataset.csv')
order_items = pd.read_csv('olist_order_items_dataset.csv')
products = pd.read_csv('olist_products_dataset.csv')
sellers = pd.read_csv('olist_sellers_dataset.csv')
geolocation = pd.read_csv('olist_geolocation_dataset.csv')

# Mesclar datasets para análise de vendas
df = pd.merge(order_items, orders, on='order_id')
df = pd.merge(df, products, on='product_id')
df = pd.merge(df, sellers, on='seller_id', how='left')

# Converter coluna de data
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
df['month'] = df['order_purchase_timestamp'].dt.to_period('M').astype(str)

In [27]:
# Vendas por mês
df['month'] = df['order_purchase_timestamp'].dt.to_period('M').astype(str)
sales_by_month = df.groupby('month')['price'].sum().reset_index()

# Vendas por categoria
sales_by_category = df.groupby('product_category_name')['price'].sum().reset_index()

In [34]:
top_categories = df.groupby('product_category_name')['order_item_id'].count().sort_values(ascending=False).head(10).reset_index()
top_categories.rename(columns={'order_item_id':'quantidade'}, inplace=True)

fig_top_categories = px.bar(top_categories, x='product_category_name', y='quantidade',
                            title='Top 10 Categorias Mais Vendidas',
                            labels={'product_category_name':'Categoria', 'quantidade':'Quantidade'},
                            template='plotly_dark')

In [36]:
top_products = df.groupby('product_id')['order_item_id'].count().sort_values(ascending=False).head(10).reset_index()
top_products.rename(columns={'order_item_id':'quantidade'}, inplace=True)

fig_top_products = px.bar(top_products, x='product_id', y='quantidade',
                          title='Top 10 Produtos Mais Vendidos (ID)',
                          labels={'product_id':'ID do Produto', 'quantidade':'Quantidade'},
                          template='plotly_dark')

In [38]:
import plotly.express as px

# Gráfico de vendas por mês
fig_month = px.line(sales_by_month, x='month', y='price',
                    title='Vendas por Mês',
                    labels={'month':'Mês', 'price':'Valor (R$)'},
                    template='plotly_dark')
fig_month.update_layout(xaxis_tickangle=-45)

# Gráfico de vendas por categoria
fig_category = px.bar(sales_by_category, x='product_category_name', y='price',
                      title='Vendas por Categoria',
                      labels={'product_category_name':'Categoria', 'price':'Valor (R$)'},
                      template='plotly_dark')
fig_category.update_layout(xaxis_tickangle=-45)


In [39]:
fig_month.show()

In [40]:
fig_category.show()

In [44]:
fig_top_categories.update_layout(xaxis_tickangle=-45)
fig_top_categories.show()

In [45]:
fig_top_products.update_layout(xaxis_tickangle=-45)
fig_top_products.show()